In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import torch
import torch.nn as nn
import torchvision
from PIL import Image
from torchvision import transforms as trn
from torch.autograd import Variable as V
import pandas as pd
device = torch.device('cuda')

In [3]:
def preprocess(image_fname):
    center_crop = trn.Compose([
        trn.Resize((256, 256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img_input = Image.open(image_fname)
    img_output = V(center_crop(img_input).unsqueeze(0))
    if torch.cuda.is_available():
      img_output = img_output.cuda()
    return img_output

In [4]:
def predict(img_path):
    img=preprocess(img_path)
    outputs = model(img)
    max = torch.softmax(outputs, 1)
    if max[0][0] > 0.5:
      masked = 'not masked'
      label = 0
      prob = max[0][0]
    else:
      masked = 'masked'
      label = 1
      prob = max[0][1]
    return label, prob, masked

In [19]:
def test(filename):
  data_path = 'gdrive/My Drive/dataset/'+filename+'/'
  test = pd.read_csv((data_path + filename+'.csv'), index_col=0)
  cor = 0
  err = 0
  for i in range(test.shape[0]):
    filename = test.iloc[i, 0]
    label = test.iloc[i, 1]
    rlt, prob, masked = predict((data_path + filename))
    print('{} predicted as {} with probability {:.3f}'.format(filename, masked, prob))
    if rlt == label:
      cor += 1.0
    else:
      err += 1.0
    AP = cor / (cor + err)
    print('Cumulative accuracy: {:.3f}'.format(AP))

**Load the model and replace the output layer with a 2 class layer. Then load the save weights from training. Set to evaluate mode and send to cuda if available.**

In [10]:
model = torchvision.models.googlenet(pretrained = True)
model.fc = nn.Linear(in_features=1024, out_features=2, bias=True)
model.load_state_dict(torch.load('gdrive/My Drive/dataset/GoogLenet_masknet.pth'))
model.eval()
#send to GPU
if torch.cuda.is_available():
  model.cuda()

 **Perform quite well with synthetic images. Accuracy: 99.5%**


In [20]:
test('test')

00169.png predicted as not masked with probability 1.000
Cumulative accuracy: 1.000
01924.png predicted as not masked with probability 1.000
Cumulative accuracy: 1.000
01470.png predicted as not masked with probability 0.999
Cumulative accuracy: 1.000
01289.png predicted as not masked with probability 1.000
Cumulative accuracy: 1.000
01250.png predicted as not masked with probability 1.000
Cumulative accuracy: 1.000
00105.png predicted as not masked with probability 0.981
Cumulative accuracy: 1.000
00435.png predicted as not masked with probability 0.996
Cumulative accuracy: 1.000
00224.png predicted as not masked with probability 1.000
Cumulative accuracy: 1.000
01142.png predicted as not masked with probability 1.000
Cumulative accuracy: 1.000
00128.png predicted as not masked with probability 1.000
Cumulative accuracy: 1.000
01148.png predicted as not masked with probability 0.998
Cumulative accuracy: 1.000
01211.png predicted as not masked with probability 1.000
Cumulative accuracy

**test on real image dataset and get accuracy of 80%.**

In [21]:
test('test_real')

im_00.jpg predicted as masked with probability 0.999
Cumulative accuracy: 1.000
im_01.jpg predicted as not masked with probability 0.992
Cumulative accuracy: 0.500
im_02.jpg predicted as masked with probability 0.599
Cumulative accuracy: 0.667
im_03.jpg predicted as masked with probability 0.997
Cumulative accuracy: 0.750
im_04.jpg predicted as masked with probability 1.000
Cumulative accuracy: 0.800
im_05.jpg predicted as not masked with probability 0.512
Cumulative accuracy: 0.667
im_06.jpg predicted as not masked with probability 0.867
Cumulative accuracy: 0.571
im_07.jpg predicted as not masked with probability 0.987
Cumulative accuracy: 0.500
im_08.jpg predicted as not masked with probability 0.976
Cumulative accuracy: 0.444
im_09.jpg predicted as masked with probability 0.849
Cumulative accuracy: 0.500
im_10.jpg predicted as masked with probability 1.000
Cumulative accuracy: 0.545
im_11.jpg predicted as masked with probability 0.504
Cumulative accuracy: 0.583
im_12.jpg predicted 